<a href="https://colab.research.google.com/github/justadudewhohacks/ipynbs/blob/master/age_recognition_comparison.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Download Data

In [0]:
!pip install -U -q PyDrive

from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
import os

train_data_json_id = '1CDMRQdAhcws_g1yDw_29ZD5DNDDyi7Xw'
test_data_json_id = '1_0dpT5HRTWocnK35KLQFDHzJiwV2-IQZ'

utk_images_7z_id = '1c61PoUhIPKeoRzB0XDI23XMDyJaCfKSh'
utk_landmarks_7z_id = '1Nxg7KKfEkDBWCqhusE1S6Edp6n3tTOuN'

appareal_labels_json_id = '1_zfGunGuqyrftDJIEKw6NVJOS55vyOrh'
appareal_images_7z_id = '1BDm6r88XLwDFsqOa2ZbbUtW1HDyHo5yA'
appareal_landmarks_7z_id = '1Am36Tk-BnjfV1d8_iUpRcW-cPfQtAN0H'

wiki_labels_json_id = '1BamAqN3tNEMh6kNQQ4C8nWf6gOA2IS6X'
wiki_images_7z_id = '1Fy3pi-Pra1IsN9HDD268nRvXa1TbsryE'
wiki_landmarks_7z_id = '1M-YeSGEEboVqNK8pTCJhbxeVaLp0TKJ4'

if not os.path.exists('./data'):
  os.makedirs('./data')
if not os.path.exists('./data/utk'):
  os.makedirs('./data/utk')
if not os.path.exists('./data/appareal'):
  os.makedirs('./data/appareal')
if not os.path.exists('./data/wiki'):
  os.makedirs('./data/wiki')

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)
    
print('downloading trainData.json and testData.json ...')
drive.CreateFile({ 'id': train_data_json_id }).GetContentFile('./data/trainData.json')
drive.CreateFile({ 'id': test_data_json_id }).GetContentFile('./data/testData.json')

print('downloading utk data ...')
drive.CreateFile({ 'id': utk_images_7z_id }).GetContentFile('./data/utk/images.7z')
drive.CreateFile({ 'id': utk_landmarks_7z_id }).GetContentFile('./data/utk/landmarks.7z')

print('downloading appareal data ...')
drive.CreateFile({ 'id': appareal_labels_json_id }).GetContentFile('./data/appareal/labels.json')
drive.CreateFile({ 'id': appareal_images_7z_id }).GetContentFile('./data/appareal/images.7z')
drive.CreateFile({ 'id': appareal_landmarks_7z_id }).GetContentFile('./data/appareal/landmarks.7z')

print('downloading wiki data ...')
drive.CreateFile({ 'id': wiki_labels_json_id }).GetContentFile('./data/wiki/labels.json')
drive.CreateFile({ 'id': wiki_images_7z_id }).GetContentFile('./data/wiki/images.7z')
drive.CreateFile({ 'id': wiki_landmarks_7z_id }).GetContentFile('./data/wiki/landmarks.7z')
  
print('unzipping data...')

!rm -rf ./sample_data
!cd ./data/utk && p7zip -d ./images.7z >> ../../utk-images.unzip.txt
!cd ./data/utk && p7zip -d ./landmarks.7z >> ../../utk-landmarks.unzip.txt
!cd ./data/appareal && p7zip -d ./images.7z >> ../../appareal-images.unzip.txt
!cd ./data/appareal && p7zip -d ./landmarks.7z >> ../../appareal-landmarks.unzip.txt
!cd ./data/wiki && p7zip -d ./images.7z >> ../../wiki-images.unzip.txt
!cd ./data/wiki && p7zip -d ./landmarks.7z >> ../../wiki-landmarks.unzip.txt
!rm -rf *.unzip.txt
print('done!')

# Common

In [0]:
!pip install git+https://github.com/justadudewhohacks/image_augment.py

In [0]:
import cv2
import math
import json
import random
import time
import os
import numpy as np
import google.colab as colab
import tensorflow as tf
from augment.augment import augment

def resize_preserve_aspect_ratio(img, size):
  height, width = img.shape[:2]
  max_dim = max(height, width)
  ratio = size / float(max_dim)
  shape = (height * ratio, width * ratio)
  resized_img = cv2.resize(img, (int(round(width * ratio)), int(round(height * ratio))))

  return resized_img

def pad_to_square(img):
  if len(img.shape) == 2:
    img = np.expand_dims(img, axis = 2)

  height, width, channels = img.shape
  max_dim = max(height, width)
  square_img = np.zeros([max_dim, max_dim, channels], dtype = img.dtype)

  dx = math.floor(abs(max_dim - width) / 2)
  dy = math.floor(abs(max_dim - height) / 2)
  square_img[dy:dy + height,dx:dx + width] = img

  return square_img

def load_json(json_file_path):
  with open(json_file_path) as json_file:  
    return json.load(json_file)

def shuffle_array(arr):
  arr_clone = arr[:]
  random.shuffle(arr_clone)
  return arr_clone
     
class BatchLoader:
  def __init__(
    self, 
    data,
    resolve_image_path,
    extract_data_labels,
    augment_image = None, 
    is_test = False,
    start_epoch = None
  ):
    if not is_test and start_epoch == None:
      raise Exception('DataLoader - start_epoch has to be defined in train mode')
    
    self.data = data
    self.resolve_image_path = resolve_image_path
    self.extract_data_labels = extract_data_labels
    self.augment_image = augment_image
    self.is_test = is_test
    self.epoch = start_epoch
    self.buffered_data = shuffle_array(self.data) if not is_test else self.data
    self.current_idx = 0
 
  def get_end_idx(self):
    return len(self.buffered_data)
   
  def load_image(self, data, image_size):
    db = data['db']
    img_file = data['file']
    file_suffix = 'chip_0' if db == 'utk' else ('face_0' if db == 'appareal' else '')
    landmarks_file = img_file.replace(file_suffix + '.jpg', file_suffix + '.json')
    landmarks_file_path = './data/' + db + '/landmarks/' + landmarks_file

    img = cv2.imread(self.resolve_image_path(data))
    if img is None:
      raise Exception('failed to read image from path: ' + img_file_path)

    if (self.augment_image is not None):
      img = self.augment_image(img, data)

    img = pad_to_square(resize_preserve_aspect_ratio(img, image_size))

    return img

  def load_image_batch(self, datas, image_size):
    preprocessed_imgs = []
    for data in datas:
      preprocessed_imgs.append(self.load_image(data, image_size))
    return np.stack(preprocessed_imgs, axis = 0)
    
  def load_labels(self, datas):
    labels = []
    for data in datas:
      labels.append(self.extract_data_labels(data))
    return np.stack(labels, axis = 0)
    
  def next_batch(self, batch_size, image_size = 112):
    if batch_size < 1:
      raise Exception('DataLoader.next_batch - invalid batch_size: ' + str(batch_size))
      
    
    from_idx = self.current_idx
    to_idx = self.current_idx + batch_size
    
    # end of epoch
    if (to_idx > len(self.buffered_data)):
      if self.is_test:
        to_idx = len(self.buffered_data)
        if to_idx == self.current_idx:
          return None
      else:
        self.epoch += 1
        self.buffered_data = self.buffered_data[from_idx:] + shuffle_array(self.data)  
        from_idx = 0
        to_idx = batch_size
      
    self.current_idx = to_idx
    
    next_data = self.buffered_data[from_idx:to_idx]
      
    batch_x = self.load_image_batch(next_data, image_size)
    batch_y = self.load_labels(next_data)
    
    return batch_x, batch_y
  

'''
--------------------------------------------------------------------------------

utility

--------------------------------------------------------------------------------
'''

def gpu_session(callback):
  config = tf.ConfigProto()
  config.gpu_options.allow_growth = True
  config.allow_soft_placement = True
  config.log_device_placement = True
  with tf.Session(config = config) as session:
    with tf.device('/gpu:0'):
      callback(session)

def get_checkpoint(epoch):
  return model_name + '.ckpt-' + str(epoch)

def download_epoch_files(start, end):
  for epoch in range(start, end):
    colab.files.download('epoch_' + str(epoch) + '.txt')
    colab.files.download(get_checkpoint(epoch) + '.index') 
    colab.files.download(get_checkpoint(epoch) + '.meta') 
    colab.files.download(get_checkpoint(epoch) + '.data-00000-of-00001')

def save_weights(var_list, checkpoint_file):
  checkpoint_data = np.array([])
  meta_data = []
  for var in var_list:
    meta_data.append({ 'shape': var.get_shape().as_list(), 'name': var.name })
    checkpoint_data = np.append(checkpoint_data, var.eval().flatten())
    
  meta_json = open(checkpoint_file + '.json', 'w')
  meta_json.write(json.dumps(meta_data))
  meta_json.close()
  np.save(checkpoint_file, checkpoint_data)


'''
--------------------------------------------------------------------------------

Data Loader

--------------------------------------------------------------------------------
'''


appareal_labels = load_json('./data/appareal/labels.json')
wiki_labels = load_json('./data/wiki/labels.json')

def extract_data_labels(data):
  db = data['db']
  img_file = data['file']

  if db == 'utk':
    age = int(float(img_file.split('_')[0]))
    return age
  elif db == 'appareal':
    age = appareal_labels[img_file]['age']
    return age
  elif db == 'wiki':
    age = wiki_labels[img_file]['age']
    return age
  else: raise('unknown db: ' + db)
    
def resolve_image_path(data):
  db = data['db']
  img_file = data['file']
  return './data/' + db + '/cropped-images/' + img_file   

def min_bbox(landmarks):
  min_x, min_y, max_x, max_y = 1.0, 1.0, 0, 0
  for pt in landmarks:
    min_x = pt['x'] if pt['x'] < min_x else min_x
    min_y = pt['y'] if pt['y'] < min_y else min_y
    max_x = max_x if pt['x'] < max_x else pt['x']
    max_y = max_y if pt['y'] < max_y else pt['y']

  return [min_x, min_y, max_x, max_y]

def augment_image(img, data):
  db = data['db']
  img_file = data['file']
  file_suffix = 'chip_0' if db == 'utk' else ('face_0' if db == 'appareal' else '')
  landmarks_file = img_file.replace(file_suffix + '.jpg', file_suffix + '.json')
  landmarks_file_path = './data/' + db + '/landmarks/' + landmarks_file

  landmarks = load_json(landmarks_file_path)
  
  
  intensity_config = { 'alpha': random.uniform(0.5, 1.5), 'beta': random.uniform(-20, 20) }
  blur_config = { 'kernel_size':  random.choice([0, 3, 5, 7, 9, 11]), 'std_dev': random.uniform(0.5, 1.5) }
  blur_prob = 0.5
  flip_prob = 0.5
  gray_prob = 0.2

  return augment(
    img,
    random_crop = min_bbox(landmarks),
    flip = random.random() < flip_prob,
    rotation_angle = random.uniform(-15, 15),
    shear = [random.uniform(0.0, 0.2), random.uniform(0.0, 0.2)],
    stretch = { 'stretch_x': random.uniform(1.0, 1.4), 'stretch_y': random.uniform(1.0, 1.4) },
    intensity = intensity_config,
    blur = blur_config if random.random() < blur_prob else None,
    hsv = [random.uniform(-5, 5), random.uniform(-15, 15), random.uniform(-20, 20)],
    to_gray = random.random() < gray_prob
    #rotation_angle = random.uniform(-5, 5),
    #blur = { 'kernel_size':  random.choice([3, 5, 7]), 'std_dev': random.uniform(0.8, 1.2) },
    #intensity = { 'alpha': random.uniform(0.8, 1.2), 'beta': random.uniform(-10, 10) },
    #to_gray = random.random() < 0.1
  )

class DataLoader(BatchLoader):
  def __init__(self, data, with_augmentation = False, start_epoch = None, is_test = False):
    BatchLoader.__init__(
      self, 
      data, 
      resolve_image_path, 
      extract_data_labels, 
      augment_image = augment_image if with_augmentation else None, 
      start_epoch = start_epoch, 
      is_test = is_test
    )

# ResNet50

https://github.com/yu4u/age-gender-estimation

In [0]:
!wget https://github.com/yu4u/age-gender-estimation/releases/download/v0.5/age_only_resnet50_weights.061-3.300-4.410.hdf5

In [33]:
from keras.applications import ResNet50, InceptionResNetV2
from keras.layers import Dense
from keras.models import Model
from keras.optimizers import SGD
from keras.callbacks import ModelCheckpoint
from keras import backend as K

def get_resnet50_model():
  base_model = ResNet50(include_top=False, weights='imagenet', input_shape=(224, 224, 3), pooling="avg")
  prediction = Dense(units=101, kernel_initializer="he_normal", use_bias=False, activation="softmax", 
                     name="pred_age")(base_model.output)
  model = Model(inputs = base_model.input, outputs = prediction)
  model.load_weights('./age_only_resnet50_weights.061-3.300-4.410.hdf5')
  model.compile(optimizer = SGD(), loss = "categorical_crossentropy")
  def predict(x):
    return model.predict(x, batch_size = x.shape[0])
  
  return predict

Using TensorFlow backend.


# Caffe Models

https://data.vision.ee.ethz.ch/cvl/rrothe/imdb-wiki/

In [0]:
!apt install -y caffe-cuda
#!apt install -y caffe-cpu

In [0]:
import caffe
caffe.set_device(0)
caffe.set_mode_gpu()

def get_caffe_model(prototxt, caffemodel):
  net = caffe.Net(prototxt, caffemodel, caffe.TEST)
  def predict(x):
    x = np.swapaxes(np.swapaxes(x, 2, 3), 1, 2)
    net.blobs['data'].data[...] = x
    return net.forward()['prob']
  
  return predict

## Real age estimation trained on IMDB-WIKI

In [0]:
!wget https://data.vision.ee.ethz.ch/cvl/rrothe/imdb-wiki/static/dex_imdb_wiki.caffemodel
!wget -O dex_imdb_wiki.prototxt https://data.vision.ee.ethz.ch/cvl/rrothe/imdb-wiki/static/age.prototxt

## Apparent age estimation trained on LAP dataset

In [0]:
!wget https://data.vision.ee.ethz.ch/cvl/rrothe/imdb-wiki/static/dex_chalearn_iccv2015.caffemodel
!wget -O dex_chalearn_iccv2015.prototxt https://data.vision.ee.ethz.ch/cvl/rrothe/imdb-wiki/static/age.prototxt

# Test

In [38]:
image_size = 224
batch_size = 1
dbs = ['utk', 'wiki', 'appareal']
test_data = load_json('./data/testData.json')

total_loss = 0
iteration_count = 0
total_loss_db = 0
iteration_count_db = 0
ts_test = time.time()

def compute_loss(pred, batch_y, is_dex = True):
  if is_dex:
    age = np.sum(pred * np.arange(101), axis = 1)
  else:
    age = np.argmax(pred, axis = 1)
     
  return np.sum(np.absolute(age - batch_y))

# dex or argmax
is_dex = True

# model

#predict = get_resnet50_model()
#predict = get_caffe_model('dex_imdb_wiki.prototxt', 'dex_imdb_wiki.caffemodel')
#predict = get_caffe_model('dex_chalearn_iccv2015.prototxt', 'dex_chalearn_iccv2015.caffemodel')

print('model loaded')

for db in dbs:
  db_data = []
  for data in test_data:
    if data['db'] == db:
      db_data.append(data)

  data_loader = DataLoader(db_data, is_test = True)
  next_batch = data_loader.next_batch(batch_size, image_size = image_size)
  
  while next_batch != None:
    batch_x, batch_y = next_batch
    pred = predict(batch_x)
    loss = compute_loss(pred, batch_y, is_dex = is_dex)
    total_loss += loss
    total_loss_db += loss
    iteration_count += 1
    iteration_count_db += 1
    next_batch = data_loader.next_batch(batch_size, image_size = image_size)

  print(str(db) + ", avg_loss= " + str(total_loss_db / iteration_count_db))
  total_loss_db = 0
  iteration_count_db = 0

print("avg_loss= " + str(total_loss / iteration_count))

model loaded
utk, avg_loss= 11.32617987261595
wiki, avg_loss= 11.568283441558872
appareal, avg_loss= 9.298751416665148
avg_loss= 11.216863203419415
